In [1]:
##This is the optimised version of the values of the IOKR algorithm with Pfam counts.
##It creates ".npy" files that can be analysed with the "analysingScores.ipynb" script
##This version takes 5 minutes for 5 subset instead of 1h and 40 min like the version stored in the "otherScripts" Folder

In [2]:
cd C:\Users\Yoo\Desktop\summerProject\hmmer-3.2.1\trial\usefulFiles\machineLearningKernelPfamFingerPrints\reducingIOKRTime

C:\Users\Yoo\Desktop\summerProject\hmmer-3.2.1\trial\usefulFiles\machineLearningKernelPfamFingerPrints\reducingIOKRTime


In [3]:
import numpy as np

In [4]:
def convertStringIntoVector (String):##Receives a binary vector in the form of a String and returns a numpy array
    intoList=list(String)
    vector=np.asarray(intoList,dtype=int)
    return vector

In [5]:
def gaussianKernelForPfam (vector1, vector2):##Does the gaussian Kernel for two vectors (pfam counts)
    gamma=0.003
    euclideanDistance=np.linalg.norm(vector1-vector2)
    kernelValue=np.exp(-gamma*(euclideanDistance**2))
    return kernelValue

In [6]:
def gaussianKernelForFingerprint (vector1, vector2):## This kernel has a different gamma used for fingerprints
    gamma=0.1
    euclideanDistance=np.linalg.norm(vector1-vector2)
    kernelValue=np.exp(-gamma*(euclideanDistance**2))
    return kernelValue

In [7]:
##Opens the two files and creates one list of Strings for pfam and another one for fingerprints
pfamFile= open ("pfamDomains0and1ForAllClusters.txt")
pfamList=[]##List of Strings
for line in pfamFile:
    line=line.strip("\n")
    pfamList.append(line[11:])
pfamFile.close()
fingerprintFile=open ("fingerprintsOnlyWithImportantSubstructures.txt")
fingerprintList=[]##List of strings
for line in fingerprintFile:
    line=line.strip("\n")
    fingerprintList.append(line[11:])
fingerprintFile.close()

In [8]:
##Converts the Lists of strings into list of Vectors
pfamVectorList=[]#List of vectors
for pfamString in pfamList:
    pfamVectorList.append(convertStringIntoVector(pfamString))
fingerprintVectorList=[]##List of vectors
for fingerprintString in fingerprintList:
    fingerprintVectorList.append(convertStringIntoVector(fingerprintString))

In [9]:
##Sets parameters for the training part of the formula (lambda*Identity*XMatrix) and inverses it
dimensionXMatrix= len (pfamVectorList)
lambdaa=0.01 #lambda seems to be a special character in python so lets put a double aa
generalXMatrix = np.zeros((dimensionXMatrix,dimensionXMatrix))
for i in range (0, dimensionXMatrix):
    for j in range (0, dimensionXMatrix):
        generalXMatrix [i,j]=gaussianKernelForPfam(pfamVectorList[i],pfamVectorList[j])#Kx with upper case
dimensionYMatrix= len (fingerprintVectorList)
generalYMatrix = np.zeros((dimensionYMatrix,dimensionYMatrix))
for i in range (0, dimensionYMatrix):
    for j in range (0, dimensionYMatrix):
        generalYMatrix [i,j]=gaussianKernelForFingerprint(fingerprintVectorList[i],fingerprintVectorList[j])#Ky with upper case

In [10]:
import random
#Creates a list of Random Numbers as big as the number of Clusters we have
#Then we will select which ones we have for each subset
listOfRandomNumbers=random.sample(range(dimensionXMatrix), dimensionXMatrix)
sampleOf206Numbers=listOfRandomNumbers[0:206]
sampleOf825Numbers= listOfRandomNumbers[206:]

In [11]:
####This is just the same script repeated several times for the 5 subsets.
####The last time, the length of the training and the test set changes, but the rest is the same

##Creates the training part of the formula once you know the specific clusters in the training set

dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the training and 206 in the test
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificXMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificXMatrix[i,j]=generalXMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificXMatrix )##Part of the formula

In [12]:
def calculatingIOKRScore (xIndex, yIndex):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=generalXMatrix[iIndex,xIndex]
        ky[i]=generalYMatrix[iIndex,yIndex]
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [13]:
score1 = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        xIndex=sampleOf206Numbers[i]
        yIndex=sampleOf206Numbers[j]
        score1[i,j]= calculatingIOKRScore(xIndex,yIndex)
np.save("score1",score1)

In [14]:
sampleOf206Numbers=listOfRandomNumbers[206:412]
sampleOf825Numbers= listOfRandomNumbers[0:206]+listOfRandomNumbers[412:]
##Creates the training part of the formula once you know the specific clusters in the training set
dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the training and 206 in the test
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificXMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificXMatrix[i,j]=generalXMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificXMatrix )##Part of the formula

In [15]:
score2 = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        xIndex=sampleOf206Numbers[i]
        yIndex=sampleOf206Numbers[j]
        score2[i,j]= calculatingIOKRScore(xIndex,yIndex)
np.save("score2",score2)

In [16]:
sampleOf206Numbers=listOfRandomNumbers[412:618]
sampleOf825Numbers= listOfRandomNumbers[0:412]+listOfRandomNumbers[618:]
##Creates the training part of the formula once you know the specific clusters in the training set
dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the training and 206 in the test
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificXMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificXMatrix[i,j]=generalXMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificXMatrix )##Part of the formula

In [17]:
score3 = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        xIndex=sampleOf206Numbers[i]
        yIndex=sampleOf206Numbers[j]
        score3[i,j]= calculatingIOKRScore(xIndex,yIndex)
np.save("score3",score3)

In [18]:
sampleOf206Numbers=listOfRandomNumbers[618:824]
sampleOf825Numbers= listOfRandomNumbers[0:618]+listOfRandomNumbers[824:]
##Creates the training part of the formula once you know the specific clusters in the training set
dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the training and 206 in the test
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificXMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificXMatrix[i,j]=generalXMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificXMatrix )##Part of the formula

In [19]:
score4 = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        xIndex=sampleOf206Numbers[i]
        yIndex=sampleOf206Numbers[j]
        score4[i,j]= calculatingIOKRScore(xIndex,yIndex)
np.save("score4",score4)

In [20]:
sampleOf206Numbers=listOfRandomNumbers[825:]
sampleOf825Numbers= listOfRandomNumbers[:825]
##Creates the training part of the formula once you know the specific clusters in the training set
dimensionTrainingSet = 824 #We divide 1031 by 5 for crossvalidation. We have 825 in the training and 206 in the test
dimensionTestSet = 207
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificXMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificXMatrix[i,j]=generalXMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificXMatrix )##Part of the formula

In [21]:
score5 = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        xIndex=sampleOf206Numbers[i]
        yIndex=sampleOf206Numbers[j]
        score5[i,j]= calculatingIOKRScore(xIndex,yIndex)
np.save("score5",score5)